In [13]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import numpy as np
from requests.exceptions import Timeout
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
import seaborn as sns
import pickle
import random

In [4]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
    client_id="ea97f40064524f9d812dfdc18ea50b86",
    client_secret="3de672dadbed45b6a26b3d0db682fc1e"))

In [5]:
playlist_1 = pd.read_csv('playlist_2.csv')

In [6]:
playlist_1

,Song ID,Song Name,Artist Name
0,6oEVnWKgPqIEPc53OYDNqG,Could've Been (feat. Bryson Tiller),H.E.R.
1,0Aa3g9EQoPNt6PiKjaUeb7,Every Kind Of Way,H.E.R.
2,29he0Joo5sz2607A6ZBT6H,Come Through (feat. Chris Brown),H.E.R.
3,6MO2bfLHKykUgCChFdw91H,"Slide (Remix) (feat. Pop Smoke, A Boogie Wit d...",H.E.R.
4,3tZs3nVjySLbL320lP4mvs,Focus,H.E.R.
...,...,...,...
23485,14KiXUcR7IDpvAhkTmqCuK,Vaya Con Dios - Digitally Remastered 1998,Tony Orlando & Dawn
23486,0mTvuMDJ3cDj0yw5j2wPEK,Candida,Tony Orlando & Dawn
23487,2asG1uvJTNu9pkTVBDL66A,Mornin' Beautiful,Tony Orlando & Dawn
23488,0o5N7SG61e0EjnPFVx3QbL,Knock Three Times,Tony Orlando & Dawn


In [7]:
playlist_df = pd.read_csv('final_clean_df.csv',index_col=0)

In [8]:
playlist_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,cluster,Song Name
0,0.450,0.352,1.0,-11.733,0.0,0.2670,0.6490,0.000004,0.2490,0.408,102.679,252747.0,4.0,3,Could've Been (feat. Bryson Tiller)
1,0.416,0.298,7.0,-8.499,1.0,0.0442,0.6350,0.000002,0.1360,0.191,95.386,160173.0,4.0,0,Every Kind Of Way
2,0.612,0.423,5.0,-8.062,0.0,0.0508,0.5880,0.000000,0.1170,0.180,135.841,214093.0,5.0,0,Come Through (feat. Chris Brown)
3,0.816,0.513,1.0,-8.363,1.0,0.2440,0.0145,0.000102,0.1400,0.170,96.981,296011.0,4.0,3,"Slide (Remix) (feat. Pop Smoke, A Boogie Wit d..."
4,0.414,0.314,1.0,-12.943,0.0,0.0548,0.2500,0.000015,0.1170,0.090,84.993,200133.0,4.0,0,Focus
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23485,0.621,0.513,5.0,-11.174,1.0,0.0511,0.4350,0.000002,0.2520,0.892,117.740,187973.0,4.0,4,Vaya Con Dios - Digitally Remastered 1998
23486,0.684,0.586,1.0,-14.119,1.0,0.0367,0.1310,0.000000,0.0378,0.896,128.534,200027.0,4.0,4,Candida
23487,0.517,0.617,0.0,-11.540,1.0,0.0359,0.1850,0.000001,0.1980,0.670,152.222,194347.0,4.0,4,Mornin' Beautiful
23488,0.794,0.577,1.0,-9.190,1.0,0.0606,0.1760,0.000000,0.1170,0.876,124.255,175533.0,4.0,4,Knock Three Times


In [9]:
top100_df = pd.read_csv('Top_100_Billboard.csv',index_col=0)

In [10]:
top100_df

,Artist,Title
0,Morgan Wallen,Last Night
1,SZA,Kill Bill
2,Miley Cyrus,Flowers
3,Eslabon Armado X Peso Pluma,Ella Baila Sola
4,Rema & Selena Gomez,Calm Down
...,...,...
95,Nate Smith,Wreckage
96,Ed Sheeran,Curtains
97,Doechii Featuring Kodak Black,What It Is (Block Boy)
98,Jack Black,Peaches


In [14]:
def check_top100(top100_df, model_file, scaler_file, playlist_df):
    song = input("Write a Song: ") # input song
    
    id_list = []   # list to store input song id
    audio_features_list = [] # list to store input song audio feature
    data_frames = [] # list to store the dataframes from the audio features list

    if song in top100_df['Title'].values:
        
        print("This song is present on top 100")  # foor loop to know if the input song is in the top 100
    else:
        client_id = 'ea97f40064524f9d812dfdc18ea50b86'
        client_secret = '3de672dadbed45b6a26b3d0db682fc1e'
        client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
        sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

        results = sp.search(q=song, limit=1, type='track')
        if len(results['tracks']['items']) > 0:
            song_id = results['tracks']['items'][0]['id']   # get the track ID
            id_list.append(song_id)   # add the track id to the list id_list
    for song_id in id_list:  # for loop to take the audio feature from the track id in the id_list
        audio_features = sp.audio_features(song_id)
        audio_features_list.append(audio_features)  # add the audio features to the list audio_features_list
        
    for item in audio_features_list: # for loop to take the audio feature from the audio_features list and transform to a dataframe
        df = pd.DataFrame(item)
        data_frames.append(df) # store the dataframes in the data_frames list
        
    final_df = pd.concat(data_frames, ignore_index=True) # concat all data frames in the data_frames list make only one data frame with all audio features
    playlist_num = final_df.select_dtypes(include='number') # take only the numeric data
    
    model_file = "KMeans.plk"
    scaler_file = "scaler_sd.pkl"
    with open(model_file, 'rb') as file:  #load the model from the pickle file
        model = pickle.load(file)
    with open(scaler_file, 'rb') as file: #load the scaler from the pickle file
        scaler = pickle.load(file)
        
        
 
        
    playlist_num_scaled = scaler.transform(playlist_num) # Apply the model and scaler to the numeric dataframe
    input_cluster = model.predict(playlist_num_scaled) # take the cluster from the input song
    
    
    matching_songs = []
    for index, cluster in enumerate(playlist_df['cluster']):  # for loop to compare the cluster from the input song with the cluster column from the playlist
        if cluster == input_cluster[0]:
            matching_songs.append(playlist_df.loc[index, 'Song Name']) # when the for loop find the matching cluster append the matching song to the list matching_songs
            
    recommended_song = random.choice(matching_songs) if matching_songs else None # find a a random song from the list and sugest a song for the same cluster to the user 
    
    
    return recommended_song

  
            

   

In [15]:
recommended_song = check_top100(top100_df, "KMeans.pkl", "scaler.pkl", playlist_df)
print("The song is not present in the top 100, we recommend this song:", recommended_song)

Write a Song: Bohemian Rhapsody
The song is not present in the top 100, we recommend this song: Wheel in the Sky
